In [2]:
# Projet d'AIKD
# XU Duo, SU Hao

# PART 1

In [3]:
# get 30 news
from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key='888d5de05649432887e6499926707525')

all_articles = newsapi.get_everything(
                                      
                                      from_param='2018-11-15',
                                      to='2018-11-20',
                                      language='en',
                                      domains='bbc.com, bbc.co.uk '
                                      )
num_news = len(all_articles['articles'])
print("get {} news".format(num_news))

get 20 news


In [4]:
test_news_X = []
test_news_date = []
for i in range(20):
    test_news_X.append(all_articles['articles'][i]['content'])
    test_news_date.append(all_articles['articles'][i]['publishedAt'])

In [5]:
# get train dataset
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train', shuffle=True)
print(twenty_train.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [6]:
# stop_word process
stop_words_I = []
with open('/Users/haosu/Desktop/停词.txt') as f:
    line = f.readline()
    while line:
        stop_words_I.append(line)
        line = f.readline()


In [7]:
#NLTK
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
def stop_word_process(text):
    stop_words = set(stopwords.words('english'))
    dataBank = []
    #Separate words
    for article in text:
        dataBank.append(word_tokenize(article))
    stop_words.add("I")
    stop_words.add(",")
    stop_words.add("(")
    stop_words.add(")")
    stop_words.add("-")
    stop_words.add("_")
    stop_words.add(".")
    stop_words.add(":")
    stop_words.add("A")
    stop_words.add("<")
    stop_words.add(">")
    stop_words.add("n't")
    stop_words.add("'s")
    stop_words.add("?")
    stop_words.add("!")
    stop_words.add("--")
    stop_words.add("chars")
    for i in range(len(stop_words_I)):
        stop_words.add(stop_words_I[i])
    # NLTK process
    twenty_train_split = []
    for i in range(len(dataBank)):
        filtered_sentence = []
        for word in dataBank[i]:
            if word not in stop_words:
                filtered_sentence.append(word)
        twenty_train_split.append(filtered_sentence)
    #twenty_train_split 是去除停词后的文本
    su = []
    for i in range(len(twenty_train_split)):
        su.append(" ".join(twenty_train_split[i]))
    return su

In [8]:
twenty_train_filtered = stop_word_process(twenty_train.data)

In [9]:
def removeNULL(text):
    # remove the null values
    for i in range(len(text)):
        if test_news_X[i] == None :
            #test_news_X.remove(text[i])
            test_news_X.pop(i)
            test_news_date.pop(i)
    return(text)

In [12]:
for i in range(len(test_news_X)):
    if isinstance(test_news_X[i],str):
        print("OK")  
    else: 
        print("NoneType")
        test_news_X = removeNULL(test_news_X)
thirty_test_filtered = stop_word_process(test_news_X)

OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK


In [13]:
# train
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train_filtered)
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
text_clf = text_clf.fit(twenty_train_filtered, twenty_train.target)

from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False), 'clf__alpha': (1e-2, 1e-3)}
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(twenty_train_filtered, twenty_train.target)
print(gs_clf.best_score_)
gs_clf.best_params_

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


0.9049849743680396


{'clf__alpha': 0.01, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}

In [14]:
# predict
X_new_counts = count_vect.transform(thirty_test_filtered)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = gs_clf.predict(thirty_test_filtered)
predict_result = []
for i in predicted:
    print(twenty_train.target_names[i])
    predict_result.append(twenty_train.target_names[i])
print("Best accuracy：%r" % (gs_clf.best_score_))

talk.politics.guns
talk.politics.guns
rec.sport.hockey
rec.motorcycles
rec.sport.hockey
sci.space
rec.motorcycles
talk.politics.misc
sci.med
sci.med
rec.sport.hockey
rec.sport.hockey
rec.sport.hockey
talk.politics.misc
rec.sport.hockey
talk.politics.guns
Best accuracy：0.9049849743680396


In [15]:
#test_news_X
#test_news_date
#predict_result
print(len(test_news_X))
print(len(test_news_date))
print(len(predict_result))
import pandas as pd
dataframe = pd.DataFrame({'content':test_news_X,'date':test_news_date,'label':predict_result})
dataframe.to_csv("/Users/haosu/Desktop/aikd.csv",index=False,sep=',')
#data = pd.read_csv('test.csv')

16
16
16


In [16]:
# KD
import uuid
print(uuid.uuid1())
'''
list1 -> ID
list3 -> label
list5 -> time
'''

20b11ea0-f7c9-11e8-b65c-acde48001122


'\nlist1 -> ID\nlist3 -> label\nlist5 -> time\n'

In [17]:
import rdflib

def create():
    g = rdflib.Graph()
    belongs_class_of = rdflib.URIRef('http://www.example.org/belongs_class_of')
    published_at = rdflib.URIRef('http://www.example.org/published_at')
    
    #sci_space = rdflib.URIRef('http://www.example.org/sci_space')
    #soc_religion_christian = rdflib.URIRef('http://www.example.org/soc_religion_christian')
    #talk_politics_mideast = rdflib.URIRef('http://www.example.org/talk_politics_mideast')
    #talk_politics_guns = rdflib.URIRef('http://www.example.org/talk_politics_guns')
    #talk_politics_misc = rdflib.URIRef('http://www.example.org/talk_politics_misc')
    #sci_med = rdflib.URIRef('http://www.example.org/sci_med')
    
    #article1 = rdflib.URIRef('http://www.example.org/article1')
    #article2 = rdflib.URIRef('http://www.example.org/article2')
    #article3 = rdflib.URIRef('http://www.example.org/article3')
    #article4 = rdflib.URIRef('http://www.example.org/article4')
    #article5 = rdflib.URIRef('http://www.example.org/article5')
    #article6 = rdflib.URIRef('http://www.example.org/article6')
    #article7 = rdflib.URIRef('http://www.example.org/article7')
    #article8 = rdflib.URIRef('http://www.example.org/article8')
    #article9 = rdflib.URIRef('http://www.example.org/article9')
    #article10 = rdflib.URIRef('http://www.example.org/article10')
    #article11 = rdflib.URIRef('http://www.example.org/article11')
    #article12 = rdflib.URIRef('http://www.example.org/article12')
    #article13 = rdflib.URIRef('http://www.example.org/article13')
    #article14 = rdflib.URIRef('http://www.example.org/article14')
    #article15 = rdflib.URIRef('http://www.example.org/article15')
    
   
    
    #def switcher(var):
     #   return {
      #      'sci_space': fun1,
       #     'soc_religion_christian': fun2,
        #    'talk_politics_mideast': fun3,
         #   'talk_politics_guns': fun4,
          #  'talk_politics_misc': fun5,
           # 'sci_med': fun6
        #}.get(var,'error')    #'error'为默认返回值，可自设置'''
    
    list1=['article1',
      'article2',
      'article3',
      'article4',
      'article5',
      'article6',
      'article7',
      'article8',
      'article9',
      'article10',
      'article11',
      'article12',
      'article13',
      'article14',
      'article15',
      'article16'
      ]
   
    
    
    list3=['sci_space',
'soc_religion_christian',
'talk_politics_mideast',
'talk_politics_guns',
'soc_religion_christian',
'talk_politics_guns',
'talk_politics_mideast',
'sci_med',
'talk_politics_guns',
'talk_politics_mideast',
'talk_politics_mideast',
'talk_politics_mideast',
'talk_politics_mideast',
'talk_politics_misc',
'talk_politics_mideast'
]
    
    list5=['20181203','20181203','20181203','20181203','20181203','20181203','20181203','20181203','20181203','20181203','20181203','20181203','20181203','20181203','20180101']
    
    list2=[]
    list4=[]
    list6=[]
    
    
    #for i in range(len(list3)):
       # list2.append(switcher(list3[i]))
        
    for i in list1:
        list2.append(rdflib.URIRef('http://www.example.org/'+i))
    '''
    ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
    '''
    for i in predict_result:
        """if i=='alt.atheism':
            list4.append(rdflib.URIRef('http://www.example.org/alt_atheism'))
        elif i=='comp.graphics':
            list4.append(rdflib.URIRef('http://www.example.org/comp.graphics'))
        elif i=='comp.os.ms-windows.misc':
            list4.append(rdflib.URIRef('http://www.example.org/comp.os.ms-windows.misc'))
        elif i=='talk_politics_guns':
            list4.append(rdflib.URIRef('http://www.example.org/talk_politics_guns'))
        elif i=='talk_politics_misc':
            list4.append(rdflib.URIRef('http://www.example.org/talk_politics_misc'))
        elif i=='sci_med':
            list4.append(rdflib.URIRef('http://www.example.org/sci_med'))
            """
        list4.append(rdflib.URIRef('http://www.example.org/'+i))
            
    for i in test_news_date:
        list6.append(rdflib.URIRef('http://www.example.org/'+i))
       
    
    #g.add((article1, belongs_class_of, sci_space))
    #print(list2)
    for i in range(len(list1)):
         g.add((list2[i],belongs_class_of,list4[i]))
    for i in range(len(list1)):       
         g.add((list2[i],published_at,list6[i]))
    ##############
    # c3,has,c4
    # c3,loc,p2
    g.serialize("graph.rdf")

In [35]:
def query():
    g = rdflib.Graph()
    g.parse("graph.rdf", format="xml")
    ################################
    # <a,?,?>
    q = "select ?article  where {?article <http://www.example.org/belongs_class_of>  <http://www.example.org/rec.sport.hockey> }"
    x = g.query(q)
    t = list(x) 
    print(len(t),'results found')
    ##### 二维
    # print(t[0][0])
    # http://www.example.org/has_border_with
    # print(t[0][1])
    # http://www.example.org/part1
    for i in range(len(t)):
        print("Result {} of {} ".format(i+1,len(t)))
        print(t[i])
    # <?,b,?>
    
if __name__ == "__main__":
    create()
    query()

6 results found
Result 1 of 6 
(rdflib.term.URIRef('http://www.example.org/article11'),)
Result 2 of 6 
(rdflib.term.URIRef('http://www.example.org/article3'),)
Result 3 of 6 
(rdflib.term.URIRef('http://www.example.org/article13'),)
Result 4 of 6 
(rdflib.term.URIRef('http://www.example.org/article5'),)
Result 5 of 6 
(rdflib.term.URIRef('http://www.example.org/article12'),)
Result 6 of 6 
(rdflib.term.URIRef('http://www.example.org/article15'),)
